# Stochastic Simulation Assignment 3


### **Contributors**  
- **Maarten Stork**  
- **Paul Jungnickel**  
- **Lucas Keijzer**

### **Overview**  
This notebook contains the code and analysis for **Assignment 3 of Stochastic Simulation**. The code follows the order specified in the assignment guidelines and replicates the experiments conducted in the referenced paper. Each section corresponds to (a) key experiment(s).

In [ ]:
from CircleParticleSim import *
import numpy as np
import numpy.random as rand
import scipy.stats

# 1)

# 2)

# 3)

In [ ]:
rand.seed(42)
run_count = 20
num_particles = 50
num_scales = 15
chain_lens = np.int64(np.round(np.logspace(1,4,num=num_scales, base=10)))
max_cooling_steps = 1000
print(chain_lens)

def run_scales(num_particles):
    arrs = np.zeros([num_scales, run_count]) 

    for i, chain_len in enumerate(chain_lens):
        num_steps = chain_len * max_cooling_steps
        # cooling_steps =  int(num_steps/chain_len)
        for j in range(run_count):
            sim  = CircleParticleSim(num_particles, steps=num_steps, seed=rand.randint(0,2**31-1),
                        cooling_schedule = paper_cooling_schedule,
                        step_size_schedule = sqrt_step_size_schedule,
                        random_step_likelihood=0.2,
                        )
            sim.run_simulation(max_cooling_steps, chain_len)
            arrs[i,j] = sim.E

        print(sim.T)

        
    np.save('data/data-3.1-{}-{}.npy'.format(num_particles, num_scales), arrs)

# for num_particles in [16,22,50,1000]:
#     run_scales(num_particles)
# run_scales(50)

num_scales = 15
arrs = np.load('data/data-3.1-50-{}.npy'.format(num_scales))
arrs = arrs[:]
scales = np.logspace(-2,1,num=num_scales, base=10)
scales1 = 100*scales[:]
scales1 = np.int64(np.round(np.logspace(1,4,num=num_scales, base=10)))
print(arrs.shape)
min_energy = np.min(arrs)
arrs -= min_energy
mean = np.mean(arrs, axis=1)
min_energy = 0


fig, ax1 = plt.subplots(figsize = (5.5,5), layout='constrained')

ax2 = ax1.twinx()

p1 = ax1.plot(scales1, mean, label='mean and range of final energies')
# # plt.plot(probs, np.percentile(arrs, 5, axis=1))
# merr = 1.96 / np.sqrt(run_count) *np.std(arrs, axis=1)
# plt.fill_between(scales1, mean - merr, mean + merr, alpha=0.3)
perc = 1
ax1.fill_between(scales1, np.percentile(arrs, perc, axis=1), np.percentile(arrs, 100-perc, axis=1), alpha=0.3)


opt_count = np.count_nonzero((np.abs(arrs - min_energy) < 1e-3), axis=1) / 20
p2 = ax2.plot(scales1, opt_count, linestyle='', marker='s', c='tab:orange', label='share of runs that reach the global minimum')
# plt.plot(scales1, min_energy*np.ones_like(scales1), linestyle = ':', color='gray', label='lowest measured energy')
ax1.set_xlabel('Markov Chain Length', fontsize=14)
# ax1.yaxis.
ax1.set_ylabel(r'$E - E_{min}$', fontsize=14)
ax1.set_yscale('log')
ax1.set_ylim([1e-6-1e12, 1e2-1e-5])
# ax2.set_ylabel('share of runs to reach the global minimum', fontsize=14)
ax2.set_ylim([0,0.99])
plt.xscale('log')
ax1.yaxis.set_tick_params(labelsize=12, labelcolor='#235b7c') 
ax2.yaxis.set_tick_params(labelsize=12, labelcolor='#E96700') 
ax1.xaxis.set_tick_params(labelsize=12, ) 
# plt.xticks(fontsize=12) 
# plt.yscale('log')
# plt.ylim([1,1.001])
ax1.legend(handles=p1+p2, loc='upper center')

fig.savefig('plots/3-50nodes-fin.pdf')

# 4)

In [ ]:
rand.seed(42)
run_count = 50
num_probs = 20
num_particles = 50
# probs = np.linspace(0,1,num_probs)

chain_len = 1000
max_cooling_steps = 300
num_steps = chain_len * max_cooling_steps

step_size_scheds = [const_step_size_schedule, random_step_size_schedule, sqrt_step_size_schedule]

def run_probs(num_particles):
    arrs = np.zeros([num_probs, run_count]) 

    for i, p in enumerate(probs):
        for j in range(run_count):
            sim  = CircleParticleSim(num_particles, steps=num_steps, seed=rand.randint(0,2**31-1),
                        cooling_schedule = paper_cooling_schedule,
                        step_size_schedule = sqrt_step_size_schedule,
                        random_step_likelihood=p
                        )
            sim.run_simulation(max_cooling_steps, chain_len)
            arrs[i,j] = sim.E

        print('.', end='')

        
    np.save('data/data-4.1-{}-{}.npy'.format(num_particles, num_probs), arrs)


# for num_particles in [30, 80, 100]:
# # # for num_particles in [16,22,50,1000]:
# # # for num_particles in [22]:
#     run_probs(num_particles)

fig, axs = plt.subplots(1,1,sharex=True, figsize=(5,5),layout='constrained')
for i, num_particles in enumerate([22, 50,  80]):
    arrs = np.load('data/data-4.1-{}-20.npy'.format(num_particles))
    arrs = arrs[:]/2
    num_probs = 20
    probs = np.linspace(0,1,num_probs)
    probs1 = probs[:]
    min_energy = np.min(arrs)
    arrs = arrs -  min_energy
    min_energy=0
    print(arrs.shape)
    mean = np.mean(arrs, axis=1)
    # plt.plot(probs1, mean, label='mean energy + IQR')
    # plt.plot(probs, np.percentile(arrs, 5, axis=1))
    # perc = 1
    # plt.fill_between(probs1, np.percentile(arrs, perc, axis=1), np.percentile(arrs, 100-perc, axis=1), alpha=0.3)
    merr = 1.96 / np.sqrt(run_count) *np.std(arrs, axis=1)
    # plt.fill_between(probs1, mean - merr, mean + merr, alpha=0.3)
    axs.errorbar(probs1, mean, merr, linestyle='', marker='o', label='{} particles'.format(num_particles), alpha=0.8)
    axs.set_yscale('log')
    # plt.plot(probs1, min_energy*np.ones_like(probs1), linestyle = ':', color='gray', label='lowest measured energy')
    # plt.ylim([2915,2925])
    # plt.savefig('plots/4-50nodes.png', dpi=600)
plt.xlabel('P(random direction)', fontsize=14)
plt.ylabel(r'$E - E_{min}$', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend()
plt.savefig('plots/4-fin.pdf')


# arrs = np.load('data/data-4.1-22-20.npy')
# arrs = arrs[:]/2
# num_probs = 20
# probs = np.linspace(0,1,num_probs)
# probs1 = probs[:]
# min_energy = np.min(arrs)
# arrs = arrs -  min_energy
# # arrs = arrs -1
# min_energy=0
# print(arrs.shape)
# mean = np.mean(arrs, axis=1)
# # plt.plot(probs1, mean, label='mean energy + IQR')
# # plt.plot(probs, np.percentile(arrs, 5, axis=1))
# # perc = 1
# # plt.fill_between(probs1, np.percentile(arrs, perc, axis=1), np.percentile(arrs, 100-perc, axis=1), alpha=0.3)
# merr = 1.96 / np.sqrt(run_count) *np.std(arrs, axis=1)
# # plt.fill_between(probs1, mean - merr, mean + merr, alpha=0.3)
# plt.errorbar(probs1, mean, merr, linestyle='', marker='o', label='energy')
# plt.plot(probs1, min_energy*np.ones_like(probs1), linestyle = ':', color='gray', label='lowest measured energy')
# plt.xlabel('P(random step)')
# plt.ylabel(r'$E - E_{min}$')
# plt.yscale('log')
# plt.legend()


In [ ]:
rand.seed(42)
run_count = 50
num_probs = 20
num_particles = 50
probs = np.linspace(0,1,num_probs)

chain_len = 1000
max_cooling_steps = 1000
num_steps = chain_len * max_cooling_steps

step_size_scheds = [const_step_size_schedule, random_step_size_schedule, sqrt_const_step_size_schedule, sqrt_step_size_schedule]

# def run_single(num_particles):
#     arrs = np.zeros([num_probs, run_count]) 

#     for i, p in enumerate(probs):
#         for j in range(run_count):
#             sim  = CircleParticleSim(num_particles, steps=num_steps, seed=rand.randint(0,2**31-1),
#                         cooling_schedule = paper_cooling_schedule,
#                         step_size_schedule = sqrt_step_size_schedule,
#                         random_step_likelihood=0.2
#                         )
#             sim.run_simulation(max_cooling_steps, chain_len)
#             arrs[i,j] = sim.E

#         print('.', end='')

        
#     np.save('data/data-4.2-{}-{}.npy'.format(num_particles, num_probs), arrs)




fig, axs = plt.subplots(1,1,sharex=True, figsize=(5,5),layout='constrained')
data = []
swap = [0,2,1,3]
for i, step_schedule in enumerate(step_size_scheds[0:]):
    arrs = np.load('data/data-4.3-{}.npy'.format(swap[i]))[0]
    # print(arrs)
    arrs = arrs/2
    arrs = arrs -  1459.5821
    # print(arrs.shape)
    data.append(arrs)

labels = ['0.1',  r'$\sqrt{T}$', 'U(0,1)',r'$U(0,1)\sqrt{T}$']

bplot = plt.boxplot(data, patch_artist=True, tick_labels=labels)
plt.yscale('log')
plt.xlabel('step size r', fontsize=14)
plt.ylabel(r'$E - E_{min}$', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('plots/4-steps.pdf')

for i in range(4):
    for j in range(i+1, 4):
        print(i, j, scipy.stats.mannwhitneyu(data[i], data[j]))